# Bra detection

## Import libraries

In [1]:
!pip install tensorflow

In [2]:
from sklearn import preprocessing 
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelBinarizer

import matplotlib.pyplot as plt
import cv2

import tensorflow as tf
from tensorflow.keras.preprocessing.image import load_img , img_to_array

import pandas as pd 
import numpy as np

import os
import time

from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.applications.mobilenet_v2 import preprocess_input

from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import MaxPooling2D
from tensorflow.keras.layers import AveragePooling2D
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Input
from tensorflow.keras.layers import Conv2D
from tensorflow.keras.layers import add
from tensorflow.keras.layers import Activation
from tensorflow.keras.layers import BatchNormalization

from tensorflow.keras.models import Sequential , Model , load_model

from tensorflow.keras.utils import to_categorical

In [3]:
def extract_label(img_path, flag):
    filename, extension = os.path.splitext(os.path.basename(img_path))
    
    subject_id, etc = filename.split("__") # 2 underscore
    bra = "without_bra"
    
    if flag:
        breast_size, waist_size, band_size, cup_size, with_bra = etc.split('_')
        
        if with_bra == 'true':
            bra = "with_bra"

        return bra
    else:
        breast_size, waist_size, band_size, cup_size = etc.split('_')
        return bra

In [4]:
bra_path = "./dataset/with_bra"
without_bra_path = "./dataset/without_bra"

img_size = 224

images = []
labels = []

In [5]:
for image in os.listdir(bra_path):
    img = cv2.imread(os.path.join(bra_path, image), cv2.IMREAD_COLOR)
    
    img_resize = cv2.resize(img, (img_size, img_size))
        
    # flag = 0 -> no bra
    # flag = 1 -> has bra
    label = extract_label(os.path.join(bra_path, image), True)
    
#     print(label)
    
    labels.append(label)
    images.append(img_resize);


In [6]:
for image in os.listdir(without_bra_path):
    img = cv2.imread(os.path.join(without_bra_path, image), cv2.IMREAD_COLOR)
    
    img_resize = cv2.resize(img, (img_size, img_size))
        
    # flag = 0 -> no bra
    # flag = 1 -> has bra
    label = extract_label(os.path.join(without_bra_path, image), False)
    
#     print(label)
    
    labels.append(label)
    images.append(img_resize);

# Performing one-hot encoding on labels

### Convert image arry to np array

In [7]:
lb = LabelBinarizer()
labels = lb.fit_transform(labels)
labels = to_categorical(labels)

In [8]:
np_images = np.array(images).astype('float32')
np_labels = np.array(labels).astype('float32')

In [9]:
print(np_images.shape)
print(np_labels.shape)

print(np_labels[1:-1])

(530, 224, 224, 3)
(1060, 1)
[[0.]
 [1.]
 [0.]
 ...
 [0.]
 [1.]
 [0.]]


In [10]:
X_train, X_val, y_train, y_val = train_test_split(np_images, np_labels, test_size = 0.3, random_state = 1)

ValueError: Found input variables with inconsistent numbers of samples: [530, 1060]

In [ ]:
img_gen = ImageDataGenerator(rotation_range=40,
                            zoom_range=0.20,
                            width_shift_range=0.2,
                            height_shift_range=0.2,
                            shear_range=0.15,
                            horizontal_flip=True,
                            fill_mode="nearest")

print("Train size: ",len(X_train),"Test size: ",len(X_val))

In [ ]:
# debug 
print(type(X_train))
print(type(y_train))

## MobileNetV2 acrchitecture

In [ ]:
model = Sequential()
model.add(Conv2D(512,(2,2),padding="same",activation="relu",input_shape=(224,224,3)))
model.add(MaxPooling2D((2,2),strides=2))

model.add(Conv2D(256,(2,2),padding="same",activation="relu"))
model.add(MaxPooling2D((2,2),strides=2))

model.add(Conv2D(128,(2,2),padding="same",activation="relu"))
model.add(MaxPooling2D((2,2),strides=2))

model.add(Conv2D(64,(2,2),padding="same",activation="relu"))
model.add(MaxPooling2D((2,2),strides=2))

model.add(Conv2D(32,(2,2),padding="same",activation="relu"))
model.add(MaxPooling2D((2,2),strides=2))

#model.add(BatchNormalization())

model.add(Dropout(0.4))
model.add(Flatten())
model.add(Dense(64,activation="relu"))
model.add(Dense(2,activation="softmax"))

model.compile(optimizer="adam",loss="binary_crossentropy",metrics=["accuracy"])
model.summary()

In [ ]:
start = time.time()

history = model.fit(
    X_train,
    y_train,
    epochs = 20,
    batch_size = 100,
    validation_data = (X_val, y_val), 
    validation_split=0.1
)

# history = model.fit(img_gen.flow(X_train,y_train,batch_size=5),
#                     steps_per_epoch=300,
#                     validation_data=(X_val,y_val),
#                     validation_steps=300,
#                     epochs=45)
end = time.time()
print("Total train time: ",(end-start)/60," mins")

In [ ]:
def plot_graph(history,string):
    plt.figure(figsize=(16,7))
    plt.plot(history.history[string],label=str(string))
    plt.plot(history.history["val_"+str(string)],label=str(string))
    plt.xlabel("Epochs")
    plt.ylabel(string)
    plt.legend([string,"val_"+string])
    plt.show()

In [ ]:
plot_graph(history,"accuracy")
plot_graph(history,"loss")